In [11]:
import pymskt as mskt
from pymskt.mesh import BoneMesh, CartilageMesh
from pymskt.mesh.anatomical import FemurACS
from pymskt.mesh.utils import get_arrow
from itkwidgets import view

import os
import SimpleITK as sitk

In [5]:
location_seg = '../../data/right_knee_example.nrrd'

In [6]:
# initiate the bone mesh object
femur = BoneMesh(
    path_seg_image=location_seg,  # path to the segmentation image being used
    label_idx=5,                  # what is the label of this bone
    list_cartilage_labels=[1]     # a list of labels for cartialge associated w/ this bone
)    

# Create the bone mesh
# This is the variance of a gaussian filter applied to binary image b4 finding the surface. 
femur.create_mesh(smooth_image_var=1.0)

# Resample the bone surface to have a specified number of nodes. 
femur.resample_surface(clusters=10000)

# Calcualte cartilage thickness for the cartialge meshes associated with the bone
# variance is for the gaussian filter applied to the binary cart meshes b4 finding the surface. 
femur.calc_cartilage_thickness(image_smooth_var_cart=0.5)

Use of `point_arrays` is deprecated. Use `point_data` instead.
Use of `cell_arrays` is deprecated. Use `cell_data` instead.


In [12]:
# Load in full seg image
seg_image = sitk.ReadImage(location_seg)
# break into sub regions. (weightbearing / trochlea / posterior condyles)
seg_image = mskt.image.cartilage_processing.get_knee_segmentation_with_femur_subregions(seg_image)

# assign femoral condyle cartilage sub regions to femur 
femur.seg_image = seg_image
femur.list_cartilage_labels=[11, 12, 13, 14, 15]
femur.assign_cartilage_regions()

# use cartilage regions to fit cylinder to condyles and create anatomic coordinate system 
femur_acs = FemurACS(femur, cart_label=(11, 12, 13, 14, 15))
femur_acs.fit()

Fitting cylinder to condyles completed successfully!


In [31]:
radius = 0.01
radius_ratio = 4
scale = 100
AP_arrow = get_arrow(femur_acs.ap_axis, scale=scale, origin=femur_acs.origin, shaft_radius=radius, tip_radius=radius*radius_ratio)
IS_arrow = get_arrow(femur_acs.is_axis, scale=scale, origin=femur_acs.origin, shaft_radius=radius, tip_radius=radius*radius_ratio)
ML_arrow_right = get_arrow(femur_acs.ml_axis, scale=scale, origin=femur_acs.origin, shaft_radius=radius, tip_radius=radius*radius_ratio)
ML_arrow_left = get_arrow(-1*femur_acs.ml_axis, scale=scale, origin=femur_acs.origin, shaft_radius=radius, tip_radius=radius*radius_ratio)

In [32]:
view(geometries=[femur.mesh, AP_arrow, IS_arrow, ML_arrow_right])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [19]:
help(get_arrow)

Help on function get_arrow in module pymskt.mesh.utils:

get_arrow(direction, origin, scale=100, tip_length=0.25, tip_radius=0.1, tip_resolution=20, shaft_radius=0.05, shaft_resolution=20)

